# (dh) 참조조문 전처리 진짜 완료.

## 데이터 로드

In [10]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from inheritance_content")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])

cursor.close()
db_connection.close()

### 참조조문 NaN 전처리

In [32]:
df['참조조문']=df['참조조문'].fillna('참조조문 없음')
print(df[df['참조조문']=='\n'].count())
# df['참조조문'].iloc[855] = '참조조문 없음'

판례정보일련번호    0
사건번호        0
선고          0
판결유형        0
판시사항        0
판결요지        0
참조조문        0
참조판례        0
판례내용        0
dtype: int64


In [33]:
print((df['참조조문'] == '\n').sum())

0


In [24]:
df['참조조문'].iloc[534]

'참조조문 없음'

In [30]:
print(df[df['판례정보일련번호']==235043])

     판례정보일련번호          사건번호  선고 판결유형 판시사항 판결요지     참조조문 참조판례  \
827    235043  2018나2006493  선고   판결   \n   \n  참조조문 없음   \n   

                                                  판례내용  
827  【원고, 항소인】 원고 1 외 2인 (소송대리인 법무법인 남산 외 6인)<br/>【...  


## 참조조문 구조 통일 (잡다한 기호 정리)

In [6]:
import re

def clean_clause(text): # 전부 , 단위로 분리될수 있게 변환

    groups=text.replace('<br/>','').replace('[증명책임]','').replace('/',',').split(',')
    groups=[group for group in groups if group.strip() != '']
    results = []

    for group in groups:
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        group=re.sub(r'(가|나|다|라|마|바|사|아|자|차|카|타|파|하)\.', '', group) # 가., 나., 다., 라., 마., 바., 사., 아., 자., 차., 카., 타., 파., 하. 댜. 형태만 제거
        group=re.sub(r'(갸|냐|댜|랴|먀|뱌|샤|야|쟈|챠|캬|탸|퍄|햐)\.', '', group) # 가나다라의 오타도 제거
        
        group=re.sub(r'[\[\［]\d+[\]\］]\s*', '', group)  # '[1]', '[2]' 형식 제거
        group = group.strip()  # 각 그룹의 양쪽 공백 제거
        
        # 쉼표로 나눠 조각을 분리
        parts = re.split(r',\s*', group)                

        transformed_parts = []

        for part in parts:    # 조, 항, 호 뒤에 문자가 붙어 있는 경우 처리

            # 조, 항, 호 뒤에 괄호와 공백 없는 문자가 있는 경우 공백 추가 (괄호 예외 처리) 
            part = re.sub(r'([0-9]+조|[0-9]+항|[0-9]+호)(?![\s\(\[])', r'\1 ', part)          
            
            part = part.strip()
            # 변환된 조문 추가
            transformed_parts.append(part)

        # 각 그룹을 쉼표로 연결하고 결과에 추가
        if transformed_parts:  # transformed_parts가 비어있지 않으면
            results.append(', '.join(transformed_parts))

    # 전체 결과를 반환
    return ', '.join(results) 

ccl=df['참조조문'].apply(clean_clause) # 함수 적용
# 공백으로 이루어진 곳 제거하기 위해 ', ,'를 ','로 치환


## 참조조문 법이름 추출

In [7]:
def extract_law_names(series):
    law_names_set=set()
    pattern = r'([가-힣]+?(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))?)'
    # 정규 표현식을 사용하여 괄호를 기준으로 분리
    
    def split_outside_parentheses(text):
        result = []
        start = 0
        stack = []  # 괄호 스택

        for i, char in enumerate(text):
            if char in "{(":
                stack.append(char)  # 괄호 열기
            elif char in "})" and stack:
                stack.pop()  # 괄호 닫기
            elif char == "," and not stack:
                # 스택이 비어 있을 때의 쉼표는 분리 기준
                result.append(text[start:i].strip())
                start = i + 1

        # 마지막 남은 부분 추가
        result.append(text[start:].strip())
        return result

    for text in series:
        # 1. 쉼표로 문자열을 분리 (괄호 안 쉼표 무시)
        parts = split_outside_parentheses(text)
        for part in parts:
            # 불필요한 \n 및 공백 제거
            part = re.sub(r'\n+', ' ', part)
            part = re.sub(r'\s+', ' ', part)  # 여러 공백을 하나로 줄임
            part = re.sub(r'[\[\]]', '', part)  # [] 괄호 제거
            part = re.sub(r'〔\d+〕', '', part)
            part = part.strip()
            part = re.sub(r'\{[^}]*\}', '', part)  # 중괄호 안의 내용 제거

            split_pattern = r'([^()]*)(\s*\([^()]*\))?'

            matches = re.findall(split_pattern, part)
            matches = [(off_maren.replace(',', '').strip(), maren.strip()) for off_maren, maren in matches]

            for match in matches:
                before_parentheses = match[0].strip()  # 괄호 이전 내용
                in_parentheses = match[1].strip() if match[1] else ''  # 괄호 내용
                # '숫자. ' 형식이 있다면 제거
                before_parentheses = re.sub(r'^\d+\.\s*', '', before_parentheses)
                # print('in: ',in_parentheses)

                # 법 이름 추출: 중괄호가 제거된 `before_parentheses`에서 패턴 찾기
                match = re.search(pattern, before_parentheses)
                if match:
                    # 패턴이 일치하는 위치까지 추출
                    law_name = before_parentheses[:match.end()] + in_parentheses
                    if len(law_name) > 1:  # 길이가 1보다 큰 경우만 추가
                        law_names_set.add(law_name)

    return list(law_names_set)


# 결과 출력
law_names = extract_law_names(ccl)
print('추출된 법 이름:', law_names)


추출된 법 이름: ['구 실용신안법(2006. 3. 3. 법률 제7872호 로 개정되기 전의 것)', '구 법인세법시행령(1985.10.5. 대통령령 제11776호 로 개정되기 전의 것)', '구 국세기본법(1998. 12. 28. 법률 제5579호 로 개정되기 전의 것)', '구 소득세법(1978.12.5 법률 제3098호 로 개정되기 전의 것)', '청원경찰법', '구 지방세법(1999. 12. 28. 법률 제6060호 로 개정되기 전의 것)', '구 조세감면규제법(1998. 12. 28. 법률 제5584호 조세특례제한법으로 전부 개정되기 전의 것)', '상속세 및 증여세법', '대일항쟁기 강제동원 피해조사 및 국외강제동원 희생자 등 지원에 관한 특별법', '구 지방공무원법(2002. 12. 18. 법률 제6786호 로 개정되기 전의 것)', '구 상속세법시행령(1996. 12. 31. 대통령령 제15193호 상속세및증여세법시행령으로 전문 개정되기 전의 것)', '구 사립학교법(2011. 4. 12. 법률 제10580호 로 개정되기 전의 것)', '구 임금채권보장법(2000. 12. 30. 법률 제6334호 로 개정되기 전의 것)', '구 소득세법시행령(1980.7.11 대통령령 제9960호 로 개정되기 전의 것)', '구 국가보위입법(법률 제3260호, 부칙 제2항 에 의하여 1981. 4. 10. 실효)', '난민법 시행령', '구 직업안정및고용촉진에관한법률(1994. 1. 7. 법률 제4733호 직업안정법으로 전문 개정되기 전의 것)', '구 공무원연금법(1979. 12. 28. 법률 제3221호 로 개정되기 전의 것)', '근로자의주거안정과목돈마련지원에관한법률', '구 노동조합 및 노동관계조정법(2006. 12. 30. 법률 제8158호 로 개정되기 전의 것)', '기초연금법 시행령', '관세법', '구 근로기준법(1997. 3. 13. 법률 제 5309호 로 제정되기 전의 것)', '해상교통안전법', '식품위생법 시행령(라)', '공공기관의 정보공

### 법 이름에서 앞에 '구'와 법 이름 뒤 괄호 제거

In [9]:
def clean_extracted_law_names(law_names):
    cleaned_law_names = []
    # 패턴 뒤의 괄호와 '구 ' 접두사 제거
    for law in law_names:
        # "제3조", "제4항", "제5호", "제6목" 등과 같은 패턴 제거
        law = re.sub(r'제\d+(조|항|호|목)\s*', '', law)
        
        # '구 ' 접두사 제거
        law = law.replace('구 ', '')
        
        # 패턴 뒤의 괄호 제거
        # 예: "폭력행위 등 처벌에 관한 법률(2014. 12. 30. 법률 제12896호로 개정되기 전의 것)" -> "폭력행위 등 처벌에 관한 법률"
        law = re.sub(r'(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', law)
        
        # 앞쪽의 마침표와 공백 제거
        law = re.sub(r'^\s*\.?\s*', '', law)        
        
        cleaned_law_names.append(law.strip())  # 최종적으로 공백 제거 후 리스트에 추가
    
    return cleaned_law_names

In [10]:
law_names_cleaned=clean_extracted_law_names(law_names)

In [11]:
law_names 

['구 실용신안법(2006. 3. 3. 법률 제7872호 로 개정되기 전의 것)',
 '구 법인세법시행령(1985.10.5. 대통령령 제11776호 로 개정되기 전의 것)',
 '구 국세기본법(1998. 12. 28. 법률 제5579호 로 개정되기 전의 것)',
 '구 소득세법(1978.12.5 법률 제3098호 로 개정되기 전의 것)',
 '청원경찰법',
 '구 지방세법(1999. 12. 28. 법률 제6060호 로 개정되기 전의 것)',
 '구 조세감면규제법(1998. 12. 28. 법률 제5584호 조세특례제한법으로 전부 개정되기 전의 것)',
 '상속세 및 증여세법',
 '대일항쟁기 강제동원 피해조사 및 국외강제동원 희생자 등 지원에 관한 특별법',
 '구 지방공무원법(2002. 12. 18. 법률 제6786호 로 개정되기 전의 것)',
 '구 상속세법시행령(1996. 12. 31. 대통령령 제15193호 상속세및증여세법시행령으로 전문 개정되기 전의 것)',
 '구 사립학교법(2011. 4. 12. 법률 제10580호 로 개정되기 전의 것)',
 '구 임금채권보장법(2000. 12. 30. 법률 제6334호 로 개정되기 전의 것)',
 '구 소득세법시행령(1980.7.11 대통령령 제9960호 로 개정되기 전의 것)',
 '구 국가보위입법(법률 제3260호, 부칙 제2항 에 의하여 1981. 4. 10. 실효)',
 '난민법 시행령',
 '구 직업안정및고용촉진에관한법률(1994. 1. 7. 법률 제4733호 직업안정법으로 전문 개정되기 전의 것)',
 '구 공무원연금법(1979. 12. 28. 법률 제3221호 로 개정되기 전의 것)',
 '근로자의주거안정과목돈마련지원에관한법률',
 '구 노동조합 및 노동관계조정법(2006. 12. 30. 법률 제8158호 로 개정되기 전의 것)',
 '기초연금법 시행령',
 '관세법',
 '구 근로기준법(1997. 3. 13. 법률 제 5309호 로 제정되기 전의 것)',
 '해상교통안전법',
 '식품위생법 시행령(라

In [12]:
law_names_cleaned

['실용신안법',
 '법인세법시행령',
 '국세기본법',
 '소득세법',
 '청원경찰법',
 '지방세법',
 '조세감면규제법',
 '상속세 및 증여세법',
 '대일항쟁기 강제동원 피해조사 및 국외강제동원 희생자 등 지원에 관한 특별법',
 '지방공무원법',
 '상속세법시행령',
 '사립학교법',
 '임금채권보장법',
 '소득세법시행령',
 '국가보위입법',
 '난민법 시행령',
 '직업안정및고용촉진에관한법률',
 '공무원연금법',
 '근로자의주거안정과목돈마련지원에관한법률',
 '노동조합 및 노동관계조정법',
 '기초연금법 시행령',
 '관세법',
 '근로기준법',
 '해상교통안전법',
 '식품위생법 시행령',
 '공공기관의 정보공개에 관한 법률',
 '식품위생법',
 '검찰청법',
 '비송사건절차법',
 '소득세법 시행령',
 '산업재해보상보험법',
 '같은법시행령',
 '공무원연금법시행령',
 '최저임금법',
 '파견근로자보호 등에 관한 법률',
 '장애인복지법',
 '교육관련기관의 정보공개에 관한 특례법',
 '기업구조조정촉진법',
 '지방세법',
 '노동조합 및 노동관계조정법',
 '공직선거 및 선거부정방지법',
 '공장 및 광업재단 저당법',
 '부정경쟁방지및영업비밀보호에관한법률',
 '직업안정법',
 '근로기준법',
 '가사소송법',
 '공유수면매립법',
 '법인세법시행령',
 '형법',
 '국민건강보험법',
 '경찰관 직무집행법',
 '국민건강보험법',
 '문화재수리 등에 관한 법률',
 '교육공무원임용령',
 '초ㆍ중등교육법 시행령',
 '법인세법',
 '산업안전보건법 시행령',
 '소득세법',
 '상법',
 '산업재해보상보험법 시행령',
 '근로기준법시행령',
 '행정절차법',
 '아시아문화중심도시 조성에 관한 특별법',
 '금융산업의 구조개선에 관한 법률',
 '택지개발촉진법',
 '상법',
 '특정범죄 가중처벌 등에 관한 법률',
 '건설산업기본법',
 '향토예비군설치법',
 '부정수표단속법',
 '최저임금법',
 '소득세법시행령',
 '계엄

In [13]:
print(len(law_names),len(law_names_cleaned),len(list(set(law_names))),len(list(set(law_names_cleaned))))

1217 1217 1217 493


In [14]:
law_dup_reduce=list(set(law_names_cleaned))

## 법 이름 붙이기

In [15]:
# 법 이름 길이가 긴 순서대로 정렬 (내림차순)
law_names = sorted(law_names, key=len, reverse=True)

In [16]:
# 법과 시행령 이름을 붙이는 함수 
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []
        
        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()
                        
            
            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                   law_name = law_name.replace('구 ', '')
        
                   # 패턴 뒤의 괄호 제거
                   law_name = re.sub(r'(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', law_name)   

                   previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정

                   part = re.sub(r'구\s*', '', part)  # 구문에서 '구 ' 제거

                   part = re.sub(r'(법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)(\([^)]*\))', r'\1', part)

                   break
            match = re.search(r'현행 (.*?) 참조', part)  # "현행"과 "참조" 사이의 내용만 추출

            if match:
               part = match.group(1) # "현행"과 "참조" 사이의 내용으로 대체
               
            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_dup_reduce):

                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    
    return x  # 문자열이 아닐 경우 그대로 반환

reason=ccl.apply(add_law_prefix)

In [17]:
reason

0       근로기준법 제28조, 근로기준법 제29조, 노동위원회법 제23조, 근로기준법 제2조...
1       파견근로자 보호 등에 관한 법률 제6조 의2 제1항, 민법 제390조, 민법 제39...
2       파견근로자 보호 등에 관한 법률 제6조 제3항(현행 제6조 의2 제1항 제3호, 제...
3       파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제...
4               근로기준법 제18조, 근로기준법 제18조, 고등교육법 시행령 제6조 제1항
                              ...                        
5011                                            소득세법 제32조
5012                   근로기준법 제28조, 근로기준법 제94조, 근로기준법 제87조
5013                             행정소송법 제12조, 귀속재산처리법 제15조
5014                  노동조합법 제33조, 노동조합법 제34조, 노동위원회규칙 제5조
5015                                             형법 제129조
Name: 참조조문, Length: 5016, dtype: object

## 참조조문 법 이름과 조 만 살려두기

In [18]:
def extract_JoandLaw(text):
    pattern=r'\b[가-힣\s]+(?:법\s*시행령|시행령|에\s*관한\s*법률|시행규칙|규칙|협정|의사록|관례|고시|대통령령|경제명령|법|령)? 제\d+조'
    matches=re.findall(pattern,text)

    return ', '.join(matches)

result=reason.apply(extract_JoandLaw)

In [19]:
result

0       근로기준법 제28조, 근로기준법 제29조, 노동위원회법 제23조, 근로기준법 제2조...
1       파견근로자 보호 등에 관한 법률 제6조, 민법 제390조, 민법 제393조, 파견근...
2                 파견근로자 보호 등에 관한 법률 제6조, 현행 제6조, 민법 제413조
3       파견근로자 보호 등에 관한 법률 제2조, 파견근로자 보호 등에 관한 법률 제2조, ...
4                   근로기준법 제18조, 근로기준법 제18조, 고등교육법 시행령 제6조
                              ...                        
5011                                            소득세법 제32조
5012                   근로기준법 제28조, 근로기준법 제94조, 근로기준법 제87조
5013                             행정소송법 제12조, 귀속재산처리법 제15조
5014                  노동조합법 제33조, 노동조합법 제34조, 노동위원회규칙 제5조
5015                                             형법 제129조
Name: 참조조문, Length: 5016, dtype: object

## 참조조문 법 이름과 조 분리

In [20]:
final=result.apply(lambda x: re.sub(r' (제\d+조)', r'/\1', x))

In [21]:
final.fillna('참조조문 없음')

0       근로기준법/제28조, 근로기준법/제29조, 노동위원회법/제23조, 근로기준법/제2조...
1       파견근로자 보호 등에 관한 법률/제6조, 민법/제390조, 민법/제393조, 파견근...
2                 파견근로자 보호 등에 관한 법률/제6조, 현행/제6조, 민법/제413조
3       파견근로자 보호 등에 관한 법률/제2조, 파견근로자 보호 등에 관한 법률/제2조, ...
4                   근로기준법/제18조, 근로기준법/제18조, 고등교육법 시행령/제6조
                              ...                        
5011                                            소득세법/제32조
5012                   근로기준법/제28조, 근로기준법/제94조, 근로기준법/제87조
5013                             행정소송법/제12조, 귀속재산처리법/제15조
5014                  노동조합법/제33조, 노동조합법/제34조, 노동위원회규칙/제5조
5015                                             형법/제129조
Name: 참조조문, Length: 5016, dtype: object

In [22]:
final

0       근로기준법/제28조, 근로기준법/제29조, 노동위원회법/제23조, 근로기준법/제2조...
1       파견근로자 보호 등에 관한 법률/제6조, 민법/제390조, 민법/제393조, 파견근...
2                 파견근로자 보호 등에 관한 법률/제6조, 현행/제6조, 민법/제413조
3       파견근로자 보호 등에 관한 법률/제2조, 파견근로자 보호 등에 관한 법률/제2조, ...
4                   근로기준법/제18조, 근로기준법/제18조, 고등교육법 시행령/제6조
                              ...                        
5011                                            소득세법/제32조
5012                   근로기준법/제28조, 근로기준법/제94조, 근로기준법/제87조
5013                             행정소송법/제12조, 귀속재산처리법/제15조
5014                  노동조합법/제33조, 노동조합법/제34조, 노동위원회규칙/제5조
5015                                             형법/제129조
Name: 참조조문, Length: 5016, dtype: object

## 데이터 프레임 작성 및 SQL 넣기

In [23]:
df_preprocess=pd.DataFrame(data=df['판례정보일련번호'])
df_preprocess['참조조문_1차전처리']=ccl
df_preprocess['참조조문_전처리']=reason
df_preprocess['참조조문_파라미터']=final
df_preprocess['참조판례_전처리']=df['참조판례']
df_preprocess.head()

,판례정보일련번호,참조조문_1차전처리,참조조문_전처리,참조조문_파라미터,참조판례_전처리
0,241221,"근로기준법 제28조, 제29조, 노동위원회법 제23조, 근로기준법 제2조 제1항 제...","근로기준법 제28조, 근로기준법 제29조, 노동위원회법 제23조, 근로기준법 제2조...","근로기준법/제28조, 근로기준법/제29조, 노동위원회법/제23조, 근로기준법/제2조...","[1] 대법원 1996. 8. 23. 선고 95누11238 판결(공1996하, 2..."
1,241233,"파견근로자 보호 등에 관한 법률 제6조 의2 제1항, 민법 제390조, 제393조,...","파견근로자 보호 등에 관한 법률 제6조 의2 제1항, 민법 제390조, 민법 제39...","파견근로자 보호 등에 관한 법률/제6조, 민법/제390조, 민법/제393조, 파견근...","[1] 대법원 2007. 11. 16. 선고 2005다3229 판결, 대법원 20..."
2,241219,구 파견근로자 보호 등에 관한 법률(2006. 12. 21. 법률 제8076호 로 ...,"파견근로자 보호 등에 관한 법률 제6조 제3항(현행 제6조 의2 제1항 제3호, 제...","파견근로자 보호 등에 관한 법률/제6조, 현행/제6조, 민법/제413조","대법원 2024. 3. 12. 선고 2019다223303, 223310 판결(공20..."
3,241051,"파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제...","파견근로자 보호 등에 관한 법률 제2조 제1호, 파견근로자 보호 등에 관한 법률 제...","파견근로자 보호 등에 관한 법률/제2조, 파견근로자 보호 등에 관한 법률/제2조, ...",[1][3] 대법원 2015. 11. 26. 선고 2013다14965 판결(공20...
4,241027,"근로기준법 제18조, 근로기준법 제18조, 구 고등교육법 시행령(2024. 2. 2...","근로기준법 제18조, 근로기준법 제18조, 고등교육법 시행령 제6조 제1항","근로기준법/제18조, 근로기준법/제18조, 고등교육법 시행령/제6조",None


In [24]:
df_preprocess.tail()

,판례정보일련번호,참조조문_1차전처리,참조조문_전처리,참조조문_파라미터,참조판례_전처리
5011,71916,소득세법(구) 제32조,소득세법 제32조,소득세법/제32조,None
5012,157410,"근로기준법 제28조, 제94조, 제87조","근로기준법 제28조, 근로기준법 제94조, 근로기준법 제87조","근로기준법/제28조, 근로기준법/제94조, 근로기준법/제87조",None
5013,133705,"행정소송법(법률 제213호 ) 제12조, 귀속재산처리법 제15조","행정소송법 제12조, 귀속재산처리법 제15조","행정소송법/제12조, 귀속재산처리법/제15조","1955. 8. 10. 선고, 4288행상41 판결(요 귀속재산처리법 제15조(15..."
5014,208998,"노동조합법 제33조, 제34조, 노동위원회규칙 제5조","노동조합법 제33조, 노동조합법 제34조, 노동위원회규칙 제5조","노동조합법/제33조, 노동조합법/제34조, 노동위원회규칙/제5조",None
5015,86243,형법 제129조,형법 제129조,형법/제129조,None


In [25]:
df_preprocess.to_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\Labor_precedent_EDA\labor_case_clause_preprocessed.csv', index=False)

In [ ]:
# db로 저장
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.dialects.mysql import LONGTEXT, TEXT
from db_info import user, password, host, port, database

####pandas dataframe에서는 불러올때 타입을 기본str로 불러와서 긴내용의 데이터삽입을 하려면 dtype을 설정해줘야함####
dtype = {
    '참조조문_1차전처리': TEXT,
    '참조조문_전처리': TEXT,
    '참조조문_파라미터': TEXT,
    '참조판례_전처리': TEXT
}
#판례내용_이유,판례내용_이유(전처리),판례내용_이유(불용어제거),클러스터,키워드

# DB정보
user = user
password = password
host = host
port = port
database = database

# CSV 파일 로드
df_labor_case_clause_preprocessed1 = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\NoF\Labor_precedent_EDA\labor_case_clause_preprocessed.csv')

# MySQL 데이터베이스 연결 설정
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 데이터프레임을 MySQL 테이블에 저장
df_labor_case_clause_preprocessed1.to_sql('labor_case_clause_preprocessed', con=engine, if_exists='replace', index=False, dtype=dtype)
# 연결 닫기
engine.dispose()

print("데이터베이스 저장 완료")